In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin

from tqdm.auto import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [4]:
x_pretrain = pd.read_csv("./pretrain_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1)
y_pretrain = pd.read_csv("./pretrain_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
x_train = pd.read_csv("./train_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1)
y_train = pd.read_csv("./train_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
x_test = pd.read_csv("./test_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1)


In [5]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 50000 to 50099
Columns: 1000 entries, feature_0000 to feature_0999
dtypes: float64(1000)
memory usage: 782.0 KB


In [7]:
x_train.describe()

,feature_0000,feature_0001,feature_0002,feature_0003,feature_0004,feature_0005,feature_0006,feature_0007,feature_0008,feature_0009,...,feature_0990,feature_0991,feature_0992,feature_0993,feature_0994,feature_0995,feature_0996,feature_0997,feature_0998,feature_0999
count,100.0,100.000000,100.000000,100.000000,100.000000,100.0,100.000000,100.00000,100.000000,100.00,...,100.00,100.0,100.00,100.0,100.0,100.000000,100.00,100.000000,100.0,100.0
mean,0.0,0.020000,0.020000,0.290000,0.310000,0.0,0.030000,0.08000,0.140000,0.01,...,0.01,0.0,0.01,0.0,0.0,0.060000,0.01,0.060000,0.0,0.0
std,0.0,0.140705,0.140705,0.456048,0.464823,0.0,0.171447,0.27266,0.348735,0.10,...,0.10,0.0,0.10,0.0,0.0,0.238683,0.10,0.238683,0.0,0.0
min,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.00,...,0.00,0.0,0.00,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0
25%,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.00,...,0.00,0.0,0.00,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0
50%,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.00,...,0.00,0.0,0.00,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0
75%,0.0,0.000000,0.000000,1.000000,1.000000,0.0,0.000000,0.00000,0.000000,0.00,...,0.00,0.0,0.00,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0
max,0.0,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.00000,1.000000,1.00,...,1.00,0.0,1.00,0.0,0.0,1.000000,1.00,1.000000,0.0,0.0


In [8]:
std = StandardScaler()
std_x_p = std.fit_transform(x_pretrain)
std_x_t = std.transform(x_train)
std_x_p = pd.DataFrame(std_x_p, columns=x_pretrain.columns)
std_x_t = pd.DataFrame(std_x_t, columns=x_train.columns)

In [10]:
std_x_p.describe()

,feature_0000,feature_0001,feature_0002,feature_0003,feature_0004,feature_0005,feature_0006,feature_0007,feature_0008,feature_0009,...,feature_0990,feature_0991,feature_0992,feature_0993,feature_0994,feature_0995,feature_0996,feature_0997,feature_0998,feature_0999
count,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,...,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04
mean,-1.861622e-17,-2.764011e-17,-3.311129e-17,-4.156675e-17,8.739676e-17,-3.517187e-17,2.714273e-17,-6.536993e-18,2.913225e-17,2.557954e-18,...,6.536993e-18,1.811884e-17,5.083045e-17,-1.666223e-17,5.329071e-18,3.591794e-17,-3.964828e-17,2.373213e-17,-1.065814e-19,4.476419e-18
std,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,...,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00
min,-7.795321e-02,-9.258979e-02,-1.775482e-01,-6.874134e-01,-6.491404e-01,-7.717030e-02,-1.496340e-01,-2.536296e-01,-3.509148e-01,-7.584792e-02,...,-9.667755e-02,-7.368391e-02,-1.127819e-01,-1.143179e-01,-3.466182e-02,-3.149077e-01,-7.103047e-02,-1.521987e-01,-1.341762e-02,-4.738174e-02
25%,-7.795321e-02,-9.258979e-02,-1.775482e-01,-6.874134e-01,-6.491404e-01,-7.717030e-02,-1.496340e-01,-2.536296e-01,-3.509148e-01,-7.584792e-02,...,-9.667755e-02,-7.368391e-02,-1.127819e-01,-1.143179e-01,-3.466182e-02,-3.149077e-01,-7.103047e-02,-1.521987e-01,-1.341762e-02,-4.738174e-02
50%,-7.795321e-02,-9.258979e-02,-1.775482e-01,-6.874134e-01,-6.491404e-01,-7.717030e-02,-1.496340e-01,-2.536296e-01,-3.509148e-01,-7.584792e-02,...,-9.667755e-02,-7.368391e-02,-1.127819e-01,-1.143179e-01,-3.466182e-02,-3.149077e-01,-7.103047e-02,-1.521987e-01,-1.341762e-02,-4.738174e-02
75%,-7.795321e-02,-9.258979e-02,-1.775482e-01,1.454729e+00,1.540499e+00,-7.717030e-02,-1.496340e-01,-2.536296e-01,-3.509148e-01,-7.584792e-02,...,-9.667755e-02,-7.368391e-02,-1.127819e-01,-1.143179e-01,-3.466182e-02,-3.149077e-01,-7.103047e-02,-1.521987e-01,-1.341762e-02,-4.738174e-02
max,1.282821e+01,1.080033e+01,5.632274e+00,1.454729e+00,1.540499e+00,1.295835e+01,6.682971e+00,3.942757e+00,2.849695e+00,1.318428e+01,...,1.034366e+01,1.357148e+01,8.866670e+00,8.747536e+00,2.885019e+01,3.175534e+00,1.407847e+01,6.570359e+00,7.452889e+01,2.110518e+01


In [11]:
std_x_t.describe()

,feature_0000,feature_0001,feature_0002,feature_0003,feature_0004,feature_0005,feature_0006,feature_0007,feature_0008,feature_0009,...,feature_0990,feature_0991,feature_0992,feature_0993,feature_0994,feature_0995,feature_0996,feature_0997,feature_0998,feature_0999
count,1.000000e+02,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000,...,100.000000,1.000000e+02,100.000000,1.000000e+02,1.000000e+02,100.000000,100.000000,100.000000,1.000000e+02,100.000000
mean,-7.795321e-02,0.125269,-0.061352,-0.066192,0.029648,-0.07717,0.055344,0.082081,0.097170,0.056753,...,0.007726,-7.368391e-02,-0.022987,-1.143179e-01,-3.466182e-02,-0.105481,0.070464,0.251155,-1.341762e-02,-0.047382
std,1.394770e-17,1.532691,0.817473,0.976920,1.017795,0.00000,1.171427,1.144187,1.116165,1.326013,...,1.044034,1.394770e-17,0.897945,1.394770e-17,6.973851e-18,0.833110,1.414950,1.604562,1.743463e-18,0.000000
min,-7.795321e-02,-0.092590,-0.177548,-0.687413,-0.649140,-0.07717,-0.149634,-0.253630,-0.350915,-0.075848,...,-0.096678,-7.368391e-02,-0.112782,-1.143179e-01,-3.466182e-02,-0.314908,-0.071030,-0.152199,-1.341762e-02,-0.047382
25%,-7.795321e-02,-0.092590,-0.177548,-0.687413,-0.649140,-0.07717,-0.149634,-0.253630,-0.350915,-0.075848,...,-0.096678,-7.368391e-02,-0.112782,-1.143179e-01,-3.466182e-02,-0.314908,-0.071030,-0.152199,-1.341762e-02,-0.047382
50%,-7.795321e-02,-0.092590,-0.177548,-0.687413,-0.649140,-0.07717,-0.149634,-0.253630,-0.350915,-0.075848,...,-0.096678,-7.368391e-02,-0.112782,-1.143179e-01,-3.466182e-02,-0.314908,-0.071030,-0.152199,-1.341762e-02,-0.047382
75%,-7.795321e-02,-0.092590,-0.177548,1.454729,1.540499,-0.07717,-0.149634,-0.253630,-0.350915,-0.075848,...,-0.096678,-7.368391e-02,-0.112782,-1.143179e-01,-3.466182e-02,-0.314908,-0.071030,-0.152199,-1.341762e-02,-0.047382
max,-7.795321e-02,10.800327,5.632274,1.454729,1.540499,-0.07717,6.682971,3.942757,2.849695,13.184278,...,10.343663,-7.368391e-02,8.866670,-1.143179e-01,-3.466182e-02,3.175534,14.078465,6.570359,-1.341762e-02,-0.047382


In [12]:
std = StandardScaler()
std_x_t = std.fit_transform(x_train)
std_x_p = std.transform(x_pretrain)
std_x_p = pd.DataFrame(std_x_p, columns=x_pretrain.columns)
std_x_t = pd.DataFrame(std_x_t, columns=x_train.columns)

In [13]:
std_x_t.describe()

,feature_0000,feature_0001,feature_0002,feature_0003,feature_0004,feature_0005,feature_0006,feature_0007,feature_0008,feature_0009,...,feature_0990,feature_0991,feature_0992,feature_0993,feature_0994,feature_0995,feature_0996,feature_0997,feature_0998,feature_0999
count,100.0,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,100.0,1.000000e+02,1.000000e+02,100.000000,1.000000e+02,...,1.000000e+02,100.0,1.000000e+02,100.0,100.0,1.000000e+02,1.000000e+02,1.000000e+02,100.0,100.0
mean,0.0,-4.829470e-17,4.996004e-18,3.774758e-17,-6.883383e-17,0.0,4.440892e-18,-5.884182e-17,0.000000,-2.248202e-17,...,-2.248202e-17,0.0,-3.913536e-17,0.0,0.0,6.661338e-17,-4.024558e-17,2.775558e-17,0.0,0.0
std,0.0,1.005038e+00,1.005038e+00,1.005038e+00,1.005038e+00,0.0,1.005038e+00,1.005038e+00,1.005038,1.005038e+00,...,1.005038e+00,0.0,1.005038e+00,0.0,0.0,1.005038e+00,1.005038e+00,1.005038e+00,0.0,0.0
min,0.0,-1.428571e-01,-1.428571e-01,-6.391015e-01,-6.702801e-01,0.0,-1.758631e-01,-2.948839e-01,-0.403473,-1.005038e-01,...,-1.005038e-01,0.0,-1.005038e-01,0.0,0.0,-2.526456e-01,-1.005038e-01,-2.526456e-01,0.0,0.0
25%,0.0,-1.428571e-01,-1.428571e-01,-6.391015e-01,-6.702801e-01,0.0,-1.758631e-01,-2.948839e-01,-0.403473,-1.005038e-01,...,-1.005038e-01,0.0,-1.005038e-01,0.0,0.0,-2.526456e-01,-1.005038e-01,-2.526456e-01,0.0,0.0
50%,0.0,-1.428571e-01,-1.428571e-01,-6.391015e-01,-6.702801e-01,0.0,-1.758631e-01,-2.948839e-01,-0.403473,-1.005038e-01,...,-1.005038e-01,0.0,-1.005038e-01,0.0,0.0,-2.526456e-01,-1.005038e-01,-2.526456e-01,0.0,0.0
75%,0.0,-1.428571e-01,-1.428571e-01,1.564697e+00,1.491914e+00,0.0,-1.758631e-01,-2.948839e-01,-0.403473,-1.005038e-01,...,-1.005038e-01,0.0,-1.005038e-01,0.0,0.0,-2.526456e-01,-1.005038e-01,-2.526456e-01,0.0,0.0
max,0.0,7.000000e+00,7.000000e+00,1.564697e+00,1.491914e+00,0.0,5.686241e+00,3.391165e+00,2.478479,9.949874e+00,...,9.949874e+00,0.0,9.949874e+00,0.0,0.0,3.958114e+00,9.949874e+00,3.958114e+00,0.0,0.0


In [14]:
std_x_p.describe()

,feature_0000,feature_0001,feature_0002,feature_0003,feature_0004,feature_0005,feature_0006,feature_0007,feature_0008,feature_0009,...,feature_0990,feature_0991,feature_0992,feature_0993,feature_0994,feature_0995,feature_0996,feature_0997,feature_0998,feature_0999
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,...,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,0.006040,-0.082143,0.075429,0.068097,-0.029276,0.005920,-0.047483,-0.072099,-0.087496,-0.043016,...,-0.007437,0.005400,0.025729,0.012900,0.001200,0.127249,-0.050051,-0.157314,0.000180,0.002240
std,0.077483,0.655741,1.229457,1.028793,0.987476,0.076714,0.857969,0.878395,0.900448,0.757947,...,0.962658,0.073287,1.119275,0.112844,0.034621,1.206381,0.710306,0.626369,0.013415,0.047276
min,0.000000,-0.142857,-0.142857,-0.639101,-0.670280,0.000000,-0.175863,-0.294884,-0.403473,-0.100504,...,-0.100504,0.000000,-0.100504,0.000000,0.000000,-0.252646,-0.100504,-0.252646,0.000000,0.000000
25%,0.000000,-0.142857,-0.142857,-0.639101,-0.670280,0.000000,-0.175863,-0.294884,-0.403473,-0.100504,...,-0.100504,0.000000,-0.100504,0.000000,0.000000,-0.252646,-0.100504,-0.252646,0.000000,0.000000
50%,0.000000,-0.142857,-0.142857,-0.639101,-0.670280,0.000000,-0.175863,-0.294884,-0.403473,-0.100504,...,-0.100504,0.000000,-0.100504,0.000000,0.000000,-0.252646,-0.100504,-0.252646,0.000000,0.000000
75%,0.000000,-0.142857,-0.142857,1.564697,1.491914,0.000000,-0.175863,-0.294884,-0.403473,-0.100504,...,-0.100504,0.000000,-0.100504,0.000000,0.000000,-0.252646,-0.100504,-0.252646,0.000000,0.000000
max,1.000000,7.000000,7.000000,1.564697,1.491914,1.000000,5.686241,3.391165,2.478479,9.949874,...,9.949874,1.000000,9.949874,1.000000,1.000000,3.958114,9.949874,3.958114,1.000000,1.000000
